In [21]:
import os

# os.chdir('../')

In [22]:
from dataclasses import dataclass
from pathlib import Path
import pandas as pd



@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [23]:
from src.utils.utlis import read_yaml,create_dir
from src.constants import *

In [24]:
class ConfigManager:
    def __init__(self,
                  config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH,
                schema_filepath = SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_dir([self.config.artifacts_root])


    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_dir([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE= config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema
        )

        return data_validation_config

In [25]:
class DataValidation:
    def __init__(self,config:DataValidationConfig):
        self.config=config

    def validate_columns(self):
        try:
            val_Status=True

            data=pd.read_csv(self.config.unzip_data_dir)
            all_cols=list(data.columns)

            all_schema=self.config.all_schema.keys()


            for col in all_cols:
                if col not in all_schema:
                    val_Status=False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {val_Status}")
                else:
                    val_Status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {val_Status}")


            return val_Status

        except Exception as e:
            raise e
                        

In [26]:
try:
    config=ConfigManager()
    data_val_config=config.get_data_validation_config()
    data_val=DataValidation(config=data_val_config)
    data_val.validate_columns()

except Exception as e:
    raise e    